<a href="https://colab.research.google.com/github/XavierCarrera/ChinookUserDataExploration/blob/main/Chinook_ExplorationAndQueries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project Setup

In [25]:
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
from google.colab import files

In [2]:
### useful: download and extract chinook sample DB
import urllib.request
import zipfile
from functools import partial
import os

chinook_url = 'http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip'
if not os.path.exists('chinook.zip'):
    print('downloading chinook.zip ', end='')
    with urllib.request.urlopen(chinook_url) as response:
        with open('chinook.zip', 'wb') as f:
            for data in iter(partial(response.read, 4*1024), b''):
                print('.', end='', flush=True)
                f.write(data)

zipfile.ZipFile('chinook.zip').extractall()
assert os.path.exists('chinook.db')

downloading chinook.zip ...........................................................................

In [3]:
engine = create_engine('sqlite:///chinook.db')
con = engine.connect()

## Data Exploration

In [4]:
table_names = engine.table_names()
print(table_names)

['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'sqlite_sequence', 'sqlite_stat1', 'tracks']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


In [5]:
for i in table_names:

  query = pd.read_sql_query(f"SELECT * FROM {i};", engine).head(3)
  query.name = i
  
  print(query.name, query.columns)
  print("--------------------------------------------------------------------------------------------------")

albums Index(['AlbumId', 'Title', 'ArtistId'], dtype='object')
--------------------------------------------------------------------------------------------------
artists Index(['ArtistId', 'Name'], dtype='object')
--------------------------------------------------------------------------------------------------
customers Index(['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City',
       'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email',
       'SupportRepId'],
      dtype='object')
--------------------------------------------------------------------------------------------------
employees Index(['EmployeeId', 'LastName', 'FirstName', 'Title', 'ReportsTo',
       'BirthDate', 'HireDate', 'Address', 'City', 'State', 'Country',
       'PostalCode', 'Phone', 'Fax', 'Email'],
      dtype='object')
--------------------------------------------------------------------------------------------------
genres Index(['GenreId', 'Name'], dtype='object')
-----------------------

In [6]:
pd.read_sql_query('''SELECT 
                      MIN(InvoiceDate),
                      MAX(InvoiceDate) 
                      FROM invoices;''', engine)

,MIN(InvoiceDate),MAX(InvoiceDate)
0,2009-01-01 00:00:00,2013-12-22 00:00:00


## Querying

In [41]:
summary_stats = pd.read_sql_query('''WITH current_year AS (
                        SELECT
                          i.CustomerId, 
                          SUM(ii.UnitPrice * ii.Quantity) AS current_revenue
                        FROM invoice_items AS ii
                        LEFT JOIN invoices AS i
                          ON ii.InvoiceId = i.InvoiceId
                        WHERE i.InvoiceDate BETWEEN '2013-01-01' AND '2013-12-01'
                        GROUP BY CustomerId),
                      last_year AS (
                        SELECT
                          SUM(ii.UnitPrice * ii.Quantity) AS last_revenue
                        FROM invoice_items AS ii
                        LEFT JOIN invoices AS i
                          ON ii.InvoiceId = i.InvoiceId
                        WHERE i.InvoiceDate BETWEEN '2012-01-01' AND '2012-12-01'
                      )

                      SELECT
                        SUM(current_revenue) as YearRevenue, 
                        ROUND(AVG(current_revenue), 2) AS ARPU,
                        ROUND(SUM(current_revenue) / last_revenue - 1, 2) AS RollingYear
                      FROM current_year, last_year;''', engine)

summary_stats.to_csv('summary_stats.csv', index=False)
summary_stats

,YearRevenue,ARPU,RollingYear
0,411.96,9.36,-0.06


In [42]:
global_users= pd.read_sql_query('''SELECT
                        c.Country,
                        COUNT(DISTINCT c.CustomerID) AS TotalUsers
                    FROM customers AS c
                    LEFT JOIN invoices AS i
                        ON c.CustomerId = i.CustomerId
                    WHERE i.InvoiceDate BETWEEN '2013-01-01' AND '2014-01-01'
                    GROUP BY c.Country''', engine)

global_users.to_csv('global_users.csv', index=False)
global_users

,Country,TotalUsers
0,Argentina,1
1,Austria,1
2,Belgium,1
3,Brazil,4
4,Canada,7
5,Czech Republic,2
6,Denmark,1
7,Finland,1
8,France,4
9,Germany,1


In [43]:
global_growth = pd.read_sql_query('''WITH users_2013 AS(
                        SELECT
                          c.Country AS country_2013,
                          COUNT(DISTINCT c.CustomerID) AS TotalUsers2013
                        FROM customers AS c
                        LEFT JOIN invoices AS i
                          ON c.CustomerId = i.CustomerId
                        WHERE i.InvoiceDate BETWEEN '2013-01-01' AND '2013-12-01'
                        GROUP BY c.Country),
                      users_2012 AS (
                        SELECT
                          c.Country,
                          COUNT(DISTINCT c.CustomerID) AS TotalUsers2012
                        FROM customers AS c
                        LEFT JOIN invoices AS i
                          ON c.CustomerId = i.CustomerId
                        WHERE i.InvoiceDate BETWEEN '2012-01-01' AND '2012-12-01'
                        GROUP BY c.Country)

                        SELECT 
                          country_2013,
                          SUM(ROUND(TotalUsers2013 / TotalUsers2012 - 1, 2)) AS YearlyGrowth
                        FROM users_2013, users_2012
                        GROUP BY country_2013;''', engine)

global_growth.to_csv('global_growth.csv', index=False)
global_growth

,country_2013,YearlyGrowth
0,Argentina,-7.0
1,Austria,-7.0
2,Belgium,-7.0
3,Brazil,39.0
4,Canada,81.0
5,Czech Republic,8.0
6,Denmark,-7.0
7,Finland,-7.0
8,France,39.0
9,Germany,-7.0


In [44]:
top_genres = pd.read_sql_query('''SELECT 
                        Genre,
                        SUM(price * quantity) AS Revenue
                      FROM (
                          SELECT 
                            g.Name AS Genre,
                            ii.UnitPrice AS price,
                            ii.Quantity AS quantity
                          FROM genres AS g
                          INNER JOIN tracks AS t 
                            ON g.GenreId = t.GenreId
                          INNER JOIN invoice_items AS ii
                            ON t.TrackId = ii.TrackId
                          INNER JOIN invoices AS i
                            ON ii.InvoiceId = i.InvoiceID
                          WHERE i.InvoiceDate >= '2013-01-01')
                        GROUP BY Genre
                        ORDER BY Revenue DESC
                        LIMIT 5''', engine)

top_genres.to_csv('top_genres.csv', index=False)
top_genres

,Genre,Revenue
0,Rock,174.24
1,Latin,79.20
2,Alternative & Punk,55.44
3,Metal,55.44
4,Jazz,21.78


In [45]:
top_artists = pd.read_sql_query('''SELECT 
                        Artist,
                        SUM(price * quantity) AS Revenue
                      FROM (
                          SELECT 
                            ar.Name AS Artist,
                            ii.UnitPrice AS price,
                            ii.Quantity AS quantity
                          FROM artists AS ar
                          INNER JOIN albums AS al
                            ON ar.ArtistId = al.ArtistId
                          INNER JOIN tracks AS t 
                            ON al.AlbumId = t.AlbumId
                          INNER JOIN invoice_items AS ii
                            ON t.TrackId = ii.TrackId
                          INNER JOIN invoices AS i
                            ON ii.InvoiceId = i.InvoiceID
                          WHERE i.InvoiceDate >= '2013-01-01')
                        GROUP BY Artist
                        ORDER BY Revenue DESC
                        LIMIT 5''', engine)

top_artists.to_csv('top_artists.csv', index=False)
top_artists

,Artist,Revenue
0,Iron Maiden,35.64
1,U2,24.75
2,Metallica,16.83
3,Lost,15.92
4,Led Zeppelin,14.85


In [46]:
quarters = pd.read_sql_query('''SELECT 
                        CASE 
                              WHEN i.InvoiceDate BETWEEN '2013-01-01' AND '2013-04-01' THEN 'Q1'
                              WHEN i.InvoiceDate BETWEEN '2013-04-01' AND '2013-07-01' THEN 'Q2'
                              WHEN i.InvoiceDate BETWEEN '2013-07-01' AND '2013-10-01' THEN 'Q3'
                              WHEN i.InvoiceDate BETWEEN '2013-10-01' AND '2014-01-01' THEN 'Q4' 
                              END AS Quarter,
                        SUM(ii.UnitPrice * ii.Quantity) AS Revenue
                      FROM invoices AS i
                      LEFT JOIN invoice_items AS ii
                        ON i.InvoiceId = ii.InvoiceId
                      GROUP BY Quarter
                      HAVING i.InvoiceDate >= '2013-01-01';''', engine)

quarters.to_csv('quarters.csv', index=False)
quarters

,Quarter,Revenue
0,Q1,102.96
1,Q2,108.90
2,Q3,112.86
3,Q4,125.86


In [47]:
active_users = pd.read_sql_query('''SELECT
                        DATE(i.InvoiceDate) as Date,
                        SUM(ii.UnitPrice * ii.Quantity) AS Revenue,
                        COUNT(DISTINCT i.CustomerId) AS ActiveUsers
                      FROM invoices AS i
                      LEFT JOIN invoice_items AS ii
                        ON i.InvoiceId = ii.InvoiceId
                      GROUP BY Date;''', engine)

active_users.to_csv('active_users.csv', index=False)
active_users

,Date,Revenue,ActiveUsers
0,2009-01-01,1.98,1
1,2009-01-02,3.96,1
2,2009-01-03,5.94,1
3,2009-01-06,8.91,1
4,2009-01-11,13.86,1
...,...,...,...
349,2013-12-05,3.96,1
350,2013-12-06,5.94,1
351,2013-12-09,8.91,1
352,2013-12-14,13.86,1


In [48]:
datasets = ['summary_stats.csv', 'global_users.csv', 'global_growth.csv',
            'top_genres.csv', 'top_artists.csv', 'quarters.csv', 'active_users.csv']

for i in datasets:
  files.download(i)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Closing Connection

In [49]:
con.close()